## Natural Language Processing with Disaster Tweets
**Predict which Tweets are about real disasters and which ones are not**

📝 Contribuição

Este repositório será continuamente atualizado com novas consultas e métodos de análise. Sugestões e colaborações são bem-vindas!

📧 Contato

Para dúvidas ou colaborações, entre em contato pelo LinkedIn ou via e-mail.

📌 Autor: Bel - Analista de Dados Técnica PL - Governo do Estado de São Paulo

# Passo 1: Importar Bibliotecas Necessárias

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Passo 2: Carregar os Dados

In [5]:
# Carregar os dados de treino e teste
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

# Visualizar as primeiras linhas dos dados de treino
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Passo 3: Análise Exploratória de Dados (EDA)

In [ ]:
# Verificar a distribuição das classes
sns.countplot(train_data['target'])
plt.title('Distribuição das Classes')
plt.show()

# Verificar valores faltantes
print(train_data.isnull().sum())

# Passo 4: Pré-processamento de Texto

In [ ]:
# Preencher valores faltantes
train_data['text'] = train_data['text'].fillna('')
test_data['text'] = test_data['text'].fillna('')

# Transformar o texto em vetores numéricos usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['text'])
X_test = vectorizer.transform(test_data['text'])

# Separar os rótulos
y_train = train_data['target']

# Passo 5: Treinar um Modelo Básico

In [ ]:
# Dividir os dados de treino em treino e validação
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Treinar o modelo
model = LogisticRegression()
model.fit(X_train_split, y_train_split)

# Prever no conjunto de validação
y_val_pred = model.predict(X_val_split)

# Avaliar o modelo usando F1-score
f1 = f1_score(y_val_split, y_val_pred)
print(f'F1 Score: {f1}')

# Passo 6: Fazer Previsões no Conjunto de Teste

In [ ]:
# Prever no conjunto de teste
test_predictions = model.predict(X_test)

# Criar o arquivo de submissão
submission = pd.DataFrame({'id': test_data['id'], 'target': test_predictions})
submission.to_csv('submission.csv', index=False)

# 🔹 Melhorias Implementadas:
Distribuição das classes (sem seaborn)

Distribuição do comprimento dos textos

Nuvem de palavras para cada classe

Gráfico interativo de palavras mais frequentes

In [ ]:
# Importar Bibliotecas Adicionais
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
from collections import Counter

# 1️⃣ Distribuição das Classes (Usando Matplotlib)
class_counts = np.bincount(train_data['target'])

plt.figure(figsize=(6, 4))
plt.barh(['Não relacionado a desastre', 'Relacionado a desastre'], class_counts, color=['blue', 'red'])
plt.xlabel('Quantidade')
plt.title('Distribuição das Classes')
plt.show()

# 2️⃣ Distribuição do Comprimento dos Textos
train_data['text_length'] = train_data['text'].apply(len)

plt.figure(figsize=(8, 5))
plt.hist(train_data['text_length'], bins=30, color='purple', alpha=0.7)
plt.xlabel('Comprimento do Texto')
plt.ylabel('Frequência')
plt.title('Distribuição do Tamanho das Mensagens')
plt.show()

# 3️⃣ Nuvem de Palavras por Classe
def gerar_nuvem_texto(texto, titulo):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(texto))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(titulo)
    plt.show()

# Criar nuvem de palavras para cada classe
gerar_nuvem_texto(train_data[train_data['target'] == 0]['text'], 'Palavras Frequentes - Não Relacionadas a Desastres')
gerar_nuvem_texto(train_data[train_data['target'] == 1]['text'], 'Palavras Frequentes - Relacionadas a Desastres')

# 4️⃣ Gráfico Interativo com Plotly - Palavras Mais Frequentes
def obter_palavras_frequentes(textos, n=20):
    palavras = ' '.join(textos).split()
    contagem = Counter(palavras)
    return contagem.most_common(n)

# Coletar palavras mais frequentes
palavras_comuns = obter_palavras_frequentes(train_data['text'])

# Converter em DataFrame (sem usar pandas)
words, counts = zip(*palavras_comuns)

# Criar gráfico interativo
fig = px.bar(x=words, y=counts, labels={'x': 'Palavras', 'y': 'Frequência'}, title='Palavras Mais Frequentes nos Tweets')
fig.show()
